In [1]:
import gensim
import pandas as pd
import nltk
import numpy as np
from tqdm import tqdm_notebook as tqdm
from pathlib import Path

In [2]:
from qiqc.preprocessors import *

In [4]:
nrows = None
datapath = Path('../input')
df_train = pd.read_csv(datapath / 'train.csv', nrows=nrows)
df_submit = pd.read_csv(datapath / 'test.csv', nrows=nrows)
texts = np.concatenate([df_train.question_text, df_submit.question_text])

preprocessor = PreprocessPipeline(
    nltk.word_tokenize,
)
transformer = TextFeatureTransformer(preprocessor)

In [5]:
%%time
transformer.fit(texts)
# transformer.vocab.filter_extremes(no_below=no_below, keep_tokens=self._reserved)
len(transformer.vocab)

CPU times: user 39.5 s, sys: 1.28 s, total: 40.8 s
Wall time: 2min 14s


In [8]:
def load_embed(file):
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    
    if file == datapath / 'embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))
        
    return embeddings_index


glove = datapath / 'embeddings/glove.840B.300d/glove.840B.300d.txt'
paragram =  datapath / 'embeddings/paragram_300_sl999/paragram_300_sl999.txt'
wiki_news = datapath / 'embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'

print("Extracting GloVe embedding")
embed_glove = load_embed(glove)
print("Extracting Paragram embedding")
embed_paragram = load_embed(paragram)
print("Extracting FastText embedding")
embed_fasttext = load_embed(wiki_news)

Extracting GloVe embedding


KeyboardInterrupt: 

In [94]:
def check_coverage(vocab, embeddings_index):
    known_words = {}
    unknown_words = {}
    nb_known_words = 0
    nb_unknown_words = 0
    for word in vocab.keys():
        try:
            known_words[word] = embeddings_index[word]
            nb_known_words += vocab[word]
        except:
            unknown_words[word] = vocab[word]
            nb_unknown_words += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(known_words) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(nb_known_words / (nb_known_words + nb_unknown_words)))
    unknown_words = sorted(unknown_words.items(), key=operator.itemgetter(1))[::-1]

    return unknown_words

{3: 1,
 0: 1,
 1: 1,
 2: 1,
 6: 272381,
 10: 37547,
 7: 156,
 13: 126,
 15: 9601,
 17: 33515,
 14: 172,
 9: 51703,
 8: 341849,
 12: 1411,
 11: 340200,
 16: 529806,
 4: 120,
 5: 1362179,
 19: 42211,
 33: 167811,
 26: 85093,
 22: 66355,
 21: 375,
 24: 3392,
 18: 172032,
 27: 27399,
 32: 51979,
 25: 397,
 29: 53999,
 31: 351507,
 20: 486,
 23: 230742,
 28: 48798,
 30: 877,
 35: 148983,
 37: 70461,
 41: 747,
 36: 4588,
 40: 23086,
 34: 24372,
 39: 3279,
 38: 221,
 44: 56,
 47: 86,
 42: 3,
 46: 13538,
 43: 7,
 45: 14,
 48: 54908,
 50: 249594,
 54: 1595,
 57: 3,
 55: 1,
 49: 584,
 58: 349,
 51: 1052,
 52: 42416,
 56: 16440,
 53: 1146,
 59: 91,
 63: 113999,
 62: 100,
 69: 366,
 66: 2321,
 60: 26,
 61: 5,
 68: 89360,
 65: 1,
 67: 195451,
 64: 584,
 70: 15916,
 71: 638,
 72: 983,
 74: 1000,
 77: 651,
 81: 48778,
 78: 318,
 73: 26948,
 75: 220845,
 79: 11152,
 76: 1323,
 80: 1942,
 89: 132513,
 85: 833,
 88: 65259,
 92: 484,
 93: 203,
 90: 93457,
 87: 61,
 91: 6731,
 83: 2,
 84: 215024,
 86: 394

In [63]:
from time import sleep

def func(sec):
    rand = np.random.uniform(low=0.0, high=1.0, size=None)
    sleep(1)
    return sec

secs = range(10)    

with ProcessPoolExecutor(max_workers=2) as executor:
    for result in tqdm(executor.map(func, secs)):
        print(result)

0
1
2
3
4
5
6
7
8
9


0.15542790587220778